In [1]:
import logging
logging.basicConfig(level=logging.INFO)

In [2]:
from cicada.communicator import SocketCommunicator

def main(communicator):
    print("Hello, World!")

SocketCommunicator.run(world_size=2, fn=main);

Hello, World!Hello, World!



In [3]:
logging.getLogger("cicada.communicator").setLevel(logging.INFO)

SocketCommunicator.run(world_size=2, fn=main);

INFO:cicada.communicator.socket.connect:Comm world player 0 listening to tcp://127.0.0.1:61481 for connections.
INFO:cicada.communicator.socket.connect:Comm world player 1 listening to tcp://127.0.0.1:61483 for connections.
INFO:cicada.communicator.socket.connect:Comm world player 0 direct connect with ['tcp://127.0.0.1:61481', 'tcp://127.0.0.1:61483'].
INFO:cicada.communicator.socket.connect:Comm world player 1 direct connect with ['tcp://127.0.0.1:61481', 'tcp://127.0.0.1:61483'].
INFO:cicada.communicator.socket.connect:Comm world player 0 tcp://127.0.0.1:61481 accepted connection from tcp://127.0.0.1:61487
INFO:cicada.communicator.socket.connect:Comm world player 1 tcp://127.0.0.1:61487 connected to player 0 tcp://127.0.0.1:61481
INFO:cicada.communicator.socket:Comm world player 0 communicator ready.
INFO:cicada.communicator.socket:Comm world player 1 communicator ready.


Hello, World!Hello, World!



INFO:cicada.communicator.socket:Comm world player 1 communicator freed.
INFO:cicada.communicator.socket:Comm world player 0 communicator freed.
INFO:cicada.communicator.socket:Comm world player 0 result: None
INFO:cicada.communicator.socket:Comm world player 1 result: None


In [4]:
logging.getLogger("cicada.communicator").setLevel(logging.WARNING)

In [5]:
from cicada.logging import Logger

def main(communicator):
    log = Logger(logger=logging.getLogger(), communicator=communicator)
    log.info("Hello, World!")

SocketCommunicator.run(world_size=2, fn=main);

INFO:root:Hello, World!
INFO:root:Hello, World!


In [6]:
def main(communicator):
    log = Logger(logger=logging.getLogger(), communicator=communicator)
    log.info(f"Hello from player {communicator.rank}!")

SocketCommunicator.run(world_size=2, fn=main);

INFO:root:Hello from player 0!
INFO:root:Hello from player 1!


In [7]:
def main(communicator):
    log = Logger(logger=logging.getLogger(), communicator=communicator)
    log.info(f"Hello from player {communicator.rank} of {communicator.world_size}!")

SocketCommunicator.run(world_size=2, fn=main);

INFO:root:Hello from player 0 of 2!
INFO:root:Hello from player 1 of 2!


In [8]:
import cicada.encoding

encoding = cicada.encoding.FixedPoint()
encoding

cicada.encoding.FixedPoint(precision=16)

In [9]:
import numpy

value = numpy.array(numpy.pi)
value

array(3.14159265)

In [10]:
import cicada.arithmetic

field = cicada.arithmetic.Field()
field

cicada.arithmetic.Field(order=18446744073709551557)

In [11]:
encoded_value = encoding.encode(value, field)
encoded_value

array(205887, dtype=object)

The encoder turns the unencoded array of real values into an array of integers with the same shape that encode the original values.  You may feel that 205887 is an unlikely way to store the value of $\pi$, but let's try decoding it to see if what we get out matches the original:

In [12]:
decoded_value = encoding.decode(encoded_value, field)
decoded_value

array(3.1415863)

You can see that the result is a value that's *pretty close* to the original, but not an exact match.  This is because the default 16 bits of precision used by the FixedPoint encoding to represent fractions can only approximate some values (of course, the original value was itself a finite approximation of $\pi$, so this shouldn't bother you too much).  For many computations 16 bits of fractional precision is more than enough, but if you need more (or less) precision, you can arrange to do so. Furthermore, Cicada provides special encodings for working with bits and boolean values, which we'll see shortly.

In [13]:
import cicada.additive

def main(communicator):
    log = Logger(logger=logging.getLogger(), communicator=communicator)
    protocol = cicada.additive.AdditiveProtocolSuite(communicator=communicator)
    
    secret = numpy.array(numpy.pi) if communicator.rank == 0 else None
    log.info(f"Player {communicator.rank} secret: {secret}")
    
    share = protocol.share(src=0, secret=secret, shape=())
    log.info(f"Player {communicator.rank} share: {share}")

SocketCommunicator.run(world_size=2, fn=main);

INFO:root:Player 0 secret: 3.141592653589793
INFO:root:Player 1 secret: None
INFO:root:Player 0 share: cicada.additive.AdditiveArrayShare(storage=8875355198802812926)
INFO:root:Player 1 share: cicada.additive.AdditiveArrayShare(storage=9571388874906944518)


In [14]:
def main(communicator):
    log = Logger(logger=logging.getLogger(), communicator=communicator)
    protocol = cicada.additive.AdditiveProtocolSuite(communicator=communicator, seed=1234)
    
    secret = numpy.array(numpy.pi) if communicator.rank == 0 else None
    log.info(f"Player {communicator.rank} secret: {secret}")
    
    share = protocol.share(src=0, secret=secret, shape=())
    log.info(f"Player {communicator.rank} share: {share}")

    revealed = protocol.reveal(share)
    log.info(f"Player {communicator.rank} revealed: {revealed}")
    
SocketCommunicator.run(world_size=2, fn=main);

INFO:root:Player 0 secret: 3.141592653589793
INFO:root:Player 1 secret: None
INFO:root:Player 0 share: cicada.additive.AdditiveArrayShare(storage=16993927038499601873)
INFO:root:Player 1 share: cicada.additive.AdditiveArrayShare(storage=1452817035210155571)
INFO:root:Player 0 revealed: 3.1415863037109375
INFO:root:Player 1 revealed: 3.1415863037109375


In [15]:
def main(communicator):
    log = Logger(logger=logging.getLogger(), communicator=communicator)
    protocol = cicada.additive.AdditiveProtocolSuite(communicator=communicator)
    
    if communicator.rank == 0:
        fortune = numpy.array(10000000)
    elif communicator.rank == 1:
        fortune = numpy.array(12000000)
    log.info(f"Player {communicator.rank} fortune: {fortune}")
 
    share0 = protocol.share(src=0, secret=fortune, shape=())
    share1 = protocol.share(src=1, secret=fortune, shape=())

    if protocol.reveal(protocol.less(share0, share1), encoding=cicada.encoding.Boolean()):
        winner = 1
    else:
        winner = 0
    
    log.info(f"Winner revealed to player {communicator.rank}: {winner}")
    
SocketCommunicator.run(world_size=2, fn=main);

INFO:root:Player 0 fortune: 10000000
INFO:root:Player 1 fortune: 12000000
INFO:root:Winner revealed to player 0: 1
INFO:root:Winner revealed to player 1: 1


In [16]:
def main(communicator):
    log = Logger(logger=logging.getLogger(), communicator=communicator)
    protocol = cicada.additive.AdditiveProtocolSuite(communicator=communicator, seed=1234)

    fortune = numpy.loadtxt(f"millionaire-{communicator.rank}.txt")

    winner = None
    winning_share = protocol.share(src=0, secret=numpy.array(0), shape=())
    for rank in communicator.ranks:
        fortune_share = protocol.share(src=rank, secret=fortune, shape=())
        less_share = protocol.less(fortune_share, winning_share)
        less = protocol.reveal(less_share, encoding=cicada.encoding.Boolean())
        if not less:
            winner = rank
            winning_share = fortune_share

    log.info(f"Winner revealed to player {communicator.rank}: {winner}")
    
SocketCommunicator.run(world_size=4, fn=main);

INFO:root:Winner revealed to player 0: 2
INFO:root:Winner revealed to player 1: 2
INFO:root:Winner revealed to player 2: 2
INFO:root:Winner revealed to player 3: 2


In this version of the program, we set aside storage for a *winning share* that starts with a value of zero.  Then, we loop over all of the players, comparing each player's shared secret with the winning share and updating it if the player's secret is larger.

If we examine the contents of the individual players' files, we see that player 2's fortune is the largest, so the choice of winner is correct:

In [17]:
for rank in range(4):
    fortune = numpy.loadtxt(f"millionaire-{rank}.txt")
    print(f"Player {rank} fortune: {fortune:>10}")

Player 0 fortune: 10000000.0
Player 1 fortune:  9000000.0
Player 2 fortune: 11000000.0
Player 3 fortune:  8500000.0
